In [1]:
import streamlit as st
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

from langchain_pinecone import PineconeVectorStore



c:\Users\Admin\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os

os.environ['PINECONE_API_KEY'] = '12f5201f-7b68-43d8-9ba9-113e6a8b90bc'

from pinecone import Pinecone

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [3]:
index_name = "medical-chatbot"
index = pc.Index(index_name)

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 7


True

In [5]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [6]:
#load the pdf files from the path
loader = DirectoryLoader('data/',glob="*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

#split text into chunks
text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks = text_splitter.split_documents(documents)

#create embeddings



FileNotFoundError: Directory not found: 'data/'

In [7]:
# download Embeddings model
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
    return embeddings

In [8]:
embeddings= download_hugging_face_embeddings()

c:\Users\Admin\anaconda3\envs\mchatbot\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
#if we have already have a index we can load it like this
docsearch=PineconeVectorStore.from_existing_index(index_name,embeddings)
query="what are heart attack"
docsearch =  docsearch.similarity_search(query, k=3)
print("result",docsearch)

result [Document(page_content='attacks a person’s own heart muscle, or myocardi-um.\nAutoantibody —An antibody that attacks the body’s\nown cells or tissues.\nMyocardial infarction —A block in the blood sup-\nply to the heart, resulting in what is commonlycalled a heart attack.\nMyocardium —The muscular middle layer of the\nheart.\nTiter —A dilution of a substance with an exact\nknown amount of fluid. For example, one part ofserum diluted with four parts of saline is a titerof 1:4.'), Document(page_content='(disordered heart beat).'), Document(page_content='can cause a heart attack if it completely blocks the\nblood flow in the heart (coronary) arteries. It can cause astroke if it completely blocks the brain (carotid) arteries.\nAtherosclerosis can also occur in the arteries of the neck,kidneys, thighs, and arms, causing kidney failure or gan-\ngrene and amputation .\nCauses and symptoms\nAtherosclerosis can begin in the late teens, but it')]


In [10]:
docsearch = PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks],
    index_name=index_name,
    embedding=embeddings
)

NameError: name 'text_chunks' is not defined

In [11]:


#create llm
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",config={'max_new_tokens':512,'temperature':0.8})


RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-682cd88a-181d86dc4999e9fb7fcc1cbb;8b836ae6-0e19-4b83-9674-c7bd180de60d)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [12]:
from langchain import PromptTemplate
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer
context:{context}
Question:{question}

Only return the helpful answer below and nothing else
Helpful answer:
"""

In [13]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt" : PROMPT}


In [14]:
from langchain.chains import RetrievalQA

In [17]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


AttributeError: 'list' object has no attribute 'as_retriever'

In [ ]:


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = ConversationalRetrievalChain.from_llm(llm=llm,chain_type='stuff',retriever=vector_store.as_retriever(search_kwargs={"k":2}),memory=memory)


In [ ]:
while True:
    user_input=input("Input Prompt:")
    result=qa.invoke({"query":user_input})
    print("Response:", result["result"])

Response: The risk of a heart attack associated with this test is low. However, if you have a history of heart disease or are at high risk for developing heart disease, your doctor may recommend additional tests to assess your cardiovascular health. It's important to follow your doctor's recommendations and take any necessary precautions to minimize your risk of complications.
Response: You don't know the answer to that question, I'm afraid.


KeyboardInterrupt: Interrupted by user

In [ ]:
st.title("HealthCare ChatBot 🧑🏽‍⚕️")
def conversation_chat(query):
    result = chain({"question": query, "chat_history": st.session_state['history']})
    st.session_state['history'].append((query, result["answer"]))
    return result["answer"]

def initialize_session_state():
    if 'history' not in st.session_state:
        st.session_state['history'] = []

    if 'generated' not in st.session_state:
        st.session_state['generated'] = ["Hello! Ask me anything about 🤗"]

    if 'past' not in st.session_state:
        st.session_state['past'] = ["Hey! 👋"]

def display_chat_history():
    reply_container = st.container()
    container = st.container()

    with container:
        with st.form(key='my_form', clear_on_submit=True):
            user_input = st.text_input("Question:", placeholder="Ask about your Mental Health", key='input')
            submit_button = st.form_submit_button(label='Send')

        if submit_button and user_input:
            output = conversation_chat(user_input)

            st.session_state['past'].append(user_input)
            st.session_state['generated'].append(output)

    if st.session_state['generated']:
        with reply_container:
            for i in range(len(st.session_state['generated'])):
                message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="thumbs")
                message(st.session_state["generated"][i], key=str(i), avatar_style="fun-emoji")
